In [19]:
spark

In [20]:
import hsfs
from hops import hdfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [28]:
node_embeddings_df = spark.read.csv("hdfs:///Projects/{}/Resources/node_embeddings.csv".format(hdfs.project_name(),inferSchema=True,header=True, mode="DROPMALFORMED"))

In [29]:
node_embeddings_df.show()

+----+------------+-----------+------------+------------+-----------+-----------+-------------+------------+-----------+-------------+-----------+-----------+------------+-----------+-------------+------------+
| _c0|         _c1|        _c2|         _c3|         _c4|        _c5|        _c6|          _c7|         _c8|        _c9|         _c10|       _c11|       _c12|        _c13|       _c14|         _c15|        _c16|
+----+------------+-----------+------------+------------+-----------+-----------+-------------+------------+-----------+-------------+-----------+-----------+------------+-----------+-------------+------------+
|null|           0|          1|           2|           3|          4|          5|            6|           7|          8|            9|         10|         11|          12|         13|           14|          15|
|7605|    0.713078|-0.70647013|   0.5298567|  -0.2180337| 0.03470659| -0.4232391|  -0.97683746|  0.29686284|-0.14874505|    0.6858924|  -0.941119| 0.6217624

In [25]:

feature_names = node_embeddings_df.columns
feature_names = ["_" + s + "c"  for s in feature_names]
feature_names[0]= 'id'
node_embeddings_node_embeddings_df = node_embeddings_df.toDF(*feature_names)

In [26]:
node_embeddings_node_embeddings_df.show()

+----+------------+-----------+------------+------------+-----------+-----------+-------------+------------+-----------+-------------+-----------+-----------+------------+-----------+-------------+------------+
|  id|       __c1c|      __c2c|       __c3c|       __c4c|      __c5c|      __c6c|        __c7c|       __c8c|      __c9c|       __c10c|     __c11c|     __c12c|      __c13c|     __c14c|       __c15c|      __c16c|
+----+------------+-----------+------------+------------+-----------+-----------+-------------+------------+-----------+-------------+-----------+-----------+------------+-----------+-------------+------------+
|null|           0|          1|           2|           3|          4|          5|            6|           7|          8|            9|         10|         11|          12|         13|           14|          15|
|7605|    0.713078|-0.70647013|   0.5298567|  -0.2180337| 0.03470659| -0.4232391|  -0.97683746|  0.29686284|-0.14874505|    0.6858924|  -0.941119| 0.6217624

In [ ]:
emb_fg = node_embeddings.join(labels, ['id'])

In [ ]:
from pyspark.sql.functions import array, coalesce      

df.select(concat(
    coalesce(col("tokens"), array()),
    coalesce(col("tokens_bigrams"), array())
)).show(truncate = False)
